In [183]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
import re
password = getpass.getpass()


········


In [184]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)


# Creating query to get the needed information

In [186]:
query='''
select c.name as 'category',f.film_id,f.rental_duration,f.rental_rate,f.length,f.rating,f.special_features ,count(sub1.rental_date) as 'rental_times'from film f 
left join inventory i using(film_id)
left join (select * from rental
where left(rental_date,7)='2005-05') sub1 using (inventory_id)
inner join film_category fc using(film_id)
inner join category c using(category_id)
group by f.film_id; '''
data=pd.read_sql_query(query, engine)
data.head()

,category,film_id,rental_duration,rental_rate,length,rating,special_features,rental_times
0,Action,19,6,0.99,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",1
1,Action,21,3,4.99,129,R,"Commentaries,Behind the Scenes",2
2,Action,29,5,2.99,168,NC-17,"Trailers,Commentaries,Deleted Scenes",2
3,Action,38,6,0.99,68,NC-17,"Trailers,Commentaries,Deleted Scenes,Behind th...",0
4,Action,56,6,2.99,129,G,"Trailers,Commentaries",1


# Creating a boolean column to judge if the films are rented last month(May) 

In [188]:
data['rental_status']=data['rental_times']>0
data.head()
data.shape

(1000, 9)

In [142]:
def rental_status(x):
    if x==True:
        x='rented'
        return x
    else:
        x='not rented'
        return x
data['rental_status']=data['rental_status'].apply(rental_status)
data.head()
        

,category,film_id,rental_duration,rental_rate,length,rating,special_features,rental_times,rental_status
0,Action,19,6,0.99,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",1,rented
1,Action,21,3,4.99,129,R,"Commentaries,Behind the Scenes",2,rented
2,Action,29,5,2.99,168,NC-17,"Trailers,Commentaries,Deleted Scenes",2,rented
3,Action,38,6,0.99,68,NC-17,"Trailers,Commentaries,Deleted Scenes,Behind th...",0,not rented
4,Action,56,6,2.99,129,G,"Trailers,Commentaries",1,rented


# Seperating 'special_features' into several subcolumns

In [143]:
sf_list = data['special_features'].unique()
sf_unique = []
for case in sf_list:
    list_of_features = re.findall("[^,]+",case)
    for feature in list_of_features:
        if feature in sf_unique:
            pass
        else:
            sf_unique.append(feature)

#This function is copied from Fernando 

In [144]:
for feature in sf_unique:
    data[feature] = data['special_features'].apply(lambda x: 1 if feature in x else 0)


In [145]:
data.head()

,category,film_id,rental_duration,rental_rate,length,rating,special_features,rental_times,rental_status,Commentaries,Deleted Scenes,Behind the Scenes,Trailers
0,Action,19,6,0.99,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",1,rented,1,1,1,0
1,Action,21,3,4.99,129,R,"Commentaries,Behind the Scenes",2,rented,1,0,1,0
2,Action,29,5,2.99,168,NC-17,"Trailers,Commentaries,Deleted Scenes",2,rented,1,1,0,1
3,Action,38,6,0.99,68,NC-17,"Trailers,Commentaries,Deleted Scenes,Behind th...",0,not rented,1,1,1,1
4,Action,56,6,2.99,129,G,"Trailers,Commentaries",1,rented,1,0,0,1


# Droping the unnecessary columns

In [146]:
data=data.drop(['rental_times','special_features'],axis=1)

In [147]:
data['rental_status'].value_counts()

rented        686
not rented    314
Name: rental_status, dtype: int64

### Checking NaN values

In [148]:
data.isna().sum()

category             0
film_id              0
rental_duration      0
rental_rate          0
length               0
rating               0
rental_status        0
Commentaries         0
Deleted Scenes       0
Behind the Scenes    0
Trailers             0
dtype: int64

# Spliting 'y' and 'x' 

In [149]:
y = data['rental_status']
X = data.drop('rental_status', axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [150]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

### normalizing the train part

In [151]:
X_train_num = X_train.select_dtypes(include = np.number)


In [152]:
transformer = MinMaxScaler().fit(X_train_num) 
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized)
X_train_norm.columns=X_train_num.columns
X_train_norm.head()

,film_id,rental_duration,rental_rate,length,Commentaries,Deleted Scenes,Behind the Scenes,Trailers
0,0.681363,0.25,1.0,0.633094,0.0,1.0,1.0,1.0
1,0.261523,0.75,1.0,0.776978,1.0,0.0,0.0,1.0
2,0.488978,0.25,0.5,0.539568,1.0,0.0,1.0,0.0
3,0.654309,0.50,1.0,0.453237,0.0,1.0,1.0,0.0
4,0.219439,0.25,0.0,0.388489,0.0,1.0,0.0,0.0


In [153]:
X_train_norm.reset_index(drop=True)

,film_id,rental_duration,rental_rate,length,Commentaries,Deleted Scenes,Behind the Scenes,Trailers
0,0.681363,0.25,1.0,0.633094,0.0,1.0,1.0,1.0
1,0.261523,0.75,1.0,0.776978,1.0,0.0,0.0,1.0
2,0.488978,0.25,0.5,0.539568,1.0,0.0,1.0,0.0
3,0.654309,0.50,1.0,0.453237,0.0,1.0,1.0,0.0
4,0.219439,0.25,0.0,0.388489,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...
795,0.371743,1.00,0.5,0.791367,0.0,0.0,1.0,1.0
796,0.696393,1.00,0.0,0.050360,0.0,1.0,1.0,1.0
797,0.856713,0.00,0.5,0.705036,1.0,1.0,1.0,0.0
798,0.468938,0.25,1.0,0.237410,1.0,1.0,0.0,1.0


In [154]:
X_train_categorical = X_train.select_dtypes(include = np.object)


C:\Users\Admin\AppData\Local\Temp\ipykernel_8984\3895947676.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train_categorical = X_train.select_dtypes(include = np.object)


In [155]:
X_train_categorical = X_train.select_dtypes(include = np.object)
X_train_cat = pd.get_dummies(X_train_categorical, 
                             columns=X_train_categorical.columns,
                             drop_first=True)
X_train_cat.isna().sum()
X_train_cat.shape

C:\Users\Admin\AppData\Local\Temp\ipykernel_8984\4179842789.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train_categorical = X_train.select_dtypes(include = np.object)


(800, 19)

In [156]:
X_train_cat.reset_index(drop=True)

,category_Animation,category_Children,category_Classics,category_Comedy,category_Documentary,category_Drama,category_Family,category_Foreign,category_Games,category_Horror,category_Music,category_New,category_Sci-Fi,category_Sports,category_Travel,rating_NC-17,rating_PG,rating_PG-13,rating_R
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
796,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
797,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
798,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [157]:
X_train_transformed = np.concatenate([X_train_norm, X_train_cat], axis=1)

In [158]:
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='multinomial').fit(X_train_transformed, y_train)

In [159]:
predictions = classification.predict(X_train_transformed)
classification.score(X_train_transformed, y_train)

0.69875

### normalizing the test part

In [160]:
X_test_num = X_test.select_dtypes(include = np.number)

In [161]:
X_test_normalized = transformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_normalized)

In [162]:
X_test_norm.columns=X_test_num.columns

In [163]:
X_test_categorical = X_test.select_dtypes(include = np.object)
X_test_cat = pd.get_dummies(X_test_categorical, 
                            columns=X_test_categorical.columns,
                            drop_first=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_8984\3812068431.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test_categorical = X_test.select_dtypes(include = np.object)


In [164]:
X_test_transformed = np.concatenate([X_test_norm, X_test_cat], axis=1)

In [165]:
predictions = classification.predict(X_test_transformed)
classification.score(X_test_transformed, y_test)

0.65

In [166]:
print(y_test.value_counts())

rented        134
not rented     66
Name: rental_status, dtype: int64


In [167]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,predictions)


array([[  5,  61],
       [  9, 125]], dtype=int64)

In [168]:
#try to use oversampling to improve this case
y_train = y_train.reset_index(drop=True) 

In [169]:
from sklearn.utils import resample

In [170]:
train = pd.concat([X_train_norm, y_train],axis=1)
train.head()


,film_id,rental_duration,rental_rate,length,Commentaries,Deleted Scenes,Behind the Scenes,Trailers,rental_status
0,0.681363,0.25,1.0,0.633094,0.0,1.0,1.0,1.0,rented
1,0.261523,0.75,1.0,0.776978,1.0,0.0,0.0,1.0,rented
2,0.488978,0.25,0.5,0.539568,1.0,0.0,1.0,0.0,not rented
3,0.654309,0.50,1.0,0.453237,0.0,1.0,1.0,0.0,not rented
4,0.219439,0.25,0.0,0.388489,0.0,1.0,0.0,0.0,not rented


In [171]:
rented_predict=train[train['rental_status']=='rented']
not_rented_predict=train[train['rental_status']=='not rented']

In [172]:
display(rented_predict.shape)
display(not_rented_predict.shape)

(552, 9)

(248, 9)

In [173]:
not_rented_predict_oversampled = resample(not_rented_predict, #<- sample from here
                                    replace=True, #<- we need replacement, since we don't have enough data otherwise
                                    n_samples = len(rented_predict)#<- make both sets the same size
                                    )

In [174]:
train_oversampled = pd.concat([rented_predict,not_rented_predict_oversampled],axis=0)
train_oversampled.tail()

,film_id,rental_duration,rental_rate,length,Commentaries,Deleted Scenes,Behind the Scenes,Trailers,rental_status
252,0.378758,1.00,0.5,0.935252,0.0,1.0,0.0,1.0,not rented
637,0.097194,1.00,0.5,0.647482,0.0,0.0,0.0,1.0,not rented
440,0.992986,0.75,0.5,0.949640,0.0,0.0,1.0,1.0,not rented
751,0.829659,1.00,0.0,0.741007,1.0,1.0,0.0,0.0,not rented
583,0.268537,1.00,0.0,0.280576,1.0,1.0,0.0,1.0,not rented


In [175]:
y_train_over = train_oversampled['rental_status'].copy()
X_train_over = train_oversampled.drop('rental_status',axis = 1).copy()

In [176]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [181]:
LR_over = LogisticRegression(random_state=0, solver='lbfgs')
LR_over.fit(X_train_over, y_train_over)
pred = LR_over.predict(X_test_norm)
LR_over.score(X_test_norm, y_test)
#print("precision: ",precision_score(y_test,pred))
#print("recall: ",recall_score(y_test,pred))
#print("f1: ",f1_score(y_test,pred))

0.54

In [190]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred)

array([[33, 33],
       [59, 75]], dtype=int64)